In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

from graph_coloring import get_konbini_oracle
from grover import grover
from utils import run_and_print_results

from graph import Graph

from graph import Konbini


def get_coloring_grover(graph: Graph, iter_count: int):
    state_size = len(graph.vertices) * graph.color_bit_length
    state_register = QuantumRegister(state_size)
    ancilla_register = QuantumRegister(graph.get_ancilla_size_needed())
    results_register = ClassicalRegister(state_size)
    qc = QuantumCircuit(state_register, ancilla_register, results_register)

    grover_instructions = grover(state_register, ancilla_register,
                                 graph.get_all_possible_state_preparing_circuit_initializer(),
                                 get_konbini_oracle(graph), iter_count)

    qc.append(grover_instructions, state_register[:] + ancilla_register[:])
    qc.measure(state_register, results_register)

    return qc

In [3]:
small_graph = Graph(4, [
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 0),
    (0, 2),
], [
    (Konbini.get_by_name('D'), 0),
    (Konbini.get_by_name('D'), 1),
])
qc = get_coloring_grover(small_graph, 1)

Graph state init: 
          ┌────────┐
q7_0: |0>┤0       ├
         │  not_D │
q7_1: |0>┤1       ├
         ├────────┤
q7_2: |0>┤0       ├
         │  not_D │
q7_3: |0>┤1       ├
         └─┬───┬──┘
q7_4: |0>──┤ H ├───
           ├───┤   
q7_5: |0>──┤ H ├───
           ├───┤   
q7_6: |0>──┤ H ├───
           ├───┤   
q7_7: |0>──┤ H ├───
           └───┘   
Grover part: 
          ┌─────────────┐     ┌─────────┐ ┌─────────────┐   ┌───┐              »
q5_0: |0>┤0            ├─────┤0        ├─┤0            ├───┤ X ├──────────────»
         │             │     │         │ │             │   ├───┤              »
q5_1: |0>┤1            ├─────┤1        ├─┤1            ├───┤ X ├──────■───────»
         │             │     │         │ │             │   ├───┤      │       »
q5_2: |0>┤2            ├─────┤2        ├─┤2            ├───┤ X ├──────┼───────»
         │             │     │         │ │             │   ├───┤      │       »
q5_3: |0>┤3            ├─────┤3        ├─┤3            ├───┤ X ├─

In [3]:
print(qc)

         ┌─────────┐ ┌─┐                     
q0_0: |0>┤0        ├─┤M├─────────────────────
         │         │ └╥┘┌─┐                  
q0_1: |0>┤1        ├──╫─┤M├──────────────────
         │         │  ║ └╥┘┌─┐               
q0_2: |0>┤2        ├──╫──╫─┤M├───────────────
         │         │  ║  ║ └╥┘┌─┐            
q0_3: |0>┤3        ├──╫──╫──╫─┤M├────────────
         │         │  ║  ║  ║ └╥┘┌─┐         
q0_4: |0>┤4        ├──╫──╫──╫──╫─┤M├─────────
         │         │  ║  ║  ║  ║ └╥┘┌─┐      
q0_5: |0>┤5        ├──╫──╫──╫──╫──╫─┤M├──────
         │         │  ║  ║  ║  ║  ║ └╥┘┌─┐   
q0_6: |0>┤6        ├──╫──╫──╫──╫──╫──╫─┤M├───
         │         │  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
q0_7: |0>┤7        ├──╫──╫──╫──╫──╫──╫──╫─┤M├
         │  Grover │  ║  ║  ║  ║  ║  ║  ║ └╥┘
q1_0: |0>┤8        ├──╫──╫──╫──╫──╫──╫──╫──╫─
         │         │  ║  ║  ║  ║  ║  ║  ║  ║ 
q1_1: |0>┤9        ├──╫──╫──╫──╫──╫──╫──╫──╫─
         │         ││ ║  ║  ║  ║  ║  ║  ║  ║ 
q1_2: |0>┤10        ├─╫──╫──╫──╫──